In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

from tkinter import *
from tkinter.ttk import *

from tkinter import *
import tkinter as tk
from tkinter import ttk
from PIL import ImageTk
import pandastable as pt

import tkinter
import customtkinter

customtkinter.set_appearance_mode("Dark")  # Modes: "System" (standard), "Dark", "Light"
customtkinter.set_default_color_theme("green")  # Themes: "blue" (standard), "green", "dark-blue"

##################### Poate sterg

conn = sqlite3.connect('database.sqlite')

def res(row):
    if row['home_team_goal'] == row['away_team_goal']:
        val = 0
    elif row['home_team_goal'] > row['away_team_goal']:
        val = 1
    else:
        val = -1
    return val

def res1(row):
    if row['away_team_goal'] == row['home_team_goal']:
        val = 0
    elif row['away_team_goal'] > row['home_team_goal']:
        val = 1
    else:
        val = -1
    return val

def autolabel(rects):
    # attach some text labels
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1*height, '%d' % int(height), ha='center', va='bottom')


countries = pd.read_sql_query("SELECT * from Country", conn)
matches = pd.read_sql_query("SELECT * from Match", conn)
leagues = pd.read_sql_query("SELECT * from League", conn)
teams = pd.read_sql_query("SELECT * from Team", conn)
tempmatch = pd.read_sql_query("SELECT * from Match", conn)

#Subsetting the five countries of interest
main_countries = ['England', 'France','Germany','Italy','Spain', 'Netherlands', 'Poland', 'Portugal', 'Scotland', 'Switzerland', 'Belgium']
countries = countries[countries.name.isin(main_countries)]
leagues = countries.merge(leagues,on='id',suffixes=('', '_y'))
seasons = matches.season.unique()
leagues

#Merge the leagues with corresponding matches
req_matches = matches[matches.league_id.isin(leagues['id'])]
req_matches = req_matches[['id','league_id','home_team_api_id','away_team_api_id','home_team_goal','away_team_goal','season']]
req_matches["total_goals"] = req_matches['home_team_goal'] + req_matches['away_team_goal']
req_matches["result"] = req_matches.apply(res,axis = 1)
req_matches.dropna(inplace=True)



#Separating the leagues for plotting and further analysis
new_matches = pd.merge(req_matches,leagues,left_on='league_id', right_on='id')
new_matches = new_matches.drop(['id_x','id_y','country_id'],axis = 1)

# HOME
matches_w_teams = pd.merge(new_matches,teams,left_on='home_team_api_id', right_on='team_api_id')
matches_w_teams = matches_w_teams.drop(['id','team_api_id','team_fifa_api_id'],axis = 1)
matches_w_teams = matches_w_teams.rename(columns={'team_long_name':'home_team_long_name','name_y':'league_name','name':'country_name'})
# matches_w_teams.head(1)

#We get teams' home win vs home loss ratio
team_home_win = matches_w_teams[matches_w_teams.result == 1].groupby('home_team_long_name').result.sum().sort_index()
team_home_loss = matches_w_teams[matches_w_teams.result ==  -1].groupby('home_team_long_name').count().result.sort_index()
team_home_draw_loss = matches_w_teams[matches_w_teams.result !=  1].groupby('home_team_long_name').count().result.sort_index()
team_home_draw_win =  matches_w_teams[matches_w_teams.result !=  -1].groupby('home_team_long_name').count().result.sort_index()
np.setdiff1d(team_home_loss.index,team_home_win.index)

#We notice that a team "SpVgg Greuther Furth" never won a home game in the one season it played in Germany. Remove it.
team_home_loss = team_home_loss[team_home_loss.index.str.contains("SpV") == False]
team_home_draw_win = team_home_draw_win[team_home_draw_win.index.str.contains("SpV") == False]

team_home_wl_ratio = team_home_win /team_home_loss
team_home_wl_ratio = team_home_wl_ratio.sort_values(ascending = False)
#print team_home_wl_ratio.head()

team_home_wld_ratio = team_home_win / team_home_draw_loss
team_home_wld_ratio = team_home_wld_ratio.sort_values(ascending = False)
#team_home_wld_ratio.head()

team_home_wdl_ratio = team_home_draw_win / team_home_loss
team_home_wdl_ratio = team_home_wdl_ratio.sort_values(ascending = False)

# AWAY
matches_w_teams1 = pd.merge(new_matches,teams,left_on='away_team_api_id', right_on='team_api_id')
matches_w_teams1 = matches_w_teams1.drop(['id','team_api_id','team_fifa_api_id'],axis = 1)
matches_w_teams1 = matches_w_teams1.rename(columns={'team_long_name':'away_team_long_name','name_y':'league_name','name':'country_name'})
# matches_w_teams1.head(1)

#We get teams' away win vs away loss ratio
team_away_win = matches_w_teams1[matches_w_teams1.result == 1].groupby('away_team_long_name').result.sum().sort_index()
team_away_loss = matches_w_teams1[matches_w_teams1.result ==  -1].groupby('away_team_long_name').count().result.sort_index()
team_away_draw_loss = matches_w_teams1[matches_w_teams1.result !=  1].groupby('away_team_long_name').count().result.sort_index()
team_away_draw_win =  matches_w_teams1[matches_w_teams1.result !=  -1].groupby('away_team_long_name').count().result.sort_index()
np.setdiff1d(team_away_loss.index,team_away_win.index)

#We notice that a team "SpVgg Greuther Furth" never won a away game in the one season it played in Germany. Remove it.
team_away_loss = team_away_loss[team_away_loss.index.str.contains("SpV") == False]
team_away_draw_win = team_away_draw_win[team_away_draw_win.index.str.contains("SpV") == False]

team_away_wl_ratio = team_away_win /team_away_loss
team_away_wl_ratio = team_away_wl_ratio.sort_values(ascending = False)
#print team_away_wl_ratio.head()

team_away_wld_ratio = team_away_win / team_away_draw_loss
team_away_wld_ratio = team_away_wld_ratio.sort_values(ascending = False)
#team_away_wld_ratio.head()

team_away_wdl_ratio = team_away_draw_win / team_away_loss
team_away_wdl_ratio = team_away_wdl_ratio.sort_values(ascending = False)

top_n = 20


################################################################################################################

#Separating the leagues for plotting and further analysis
new_matches = pd.merge(req_matches,leagues,left_on='league_id', right_on='id')
new_matches = new_matches.drop(['id_x','id_y','country_id'],axis = 1)

english = new_matches[new_matches.name == "England"]
french = new_matches[new_matches.name == "France"]
italian = new_matches[new_matches.name == "Italy"]
spanish = new_matches[new_matches.name == "Spain"]
german = new_matches[new_matches.name == "Germany"]
dutch = new_matches[new_matches.name == "Netherlands"]
polish = new_matches[new_matches.name == "Poland"]
portuguese = new_matches[new_matches.name == "Portugal"]
scots = new_matches[new_matches.name == "Scotland"]
swit = new_matches[new_matches.name == "Switzerland"]
belg = new_matches[new_matches.name == "Belgium"]

# sum_goals = new_group_matches.home_team_goal.sum()
e = english.groupby('season')
f = french.groupby('season')
i = italian.groupby('season')
s = spanish.groupby('season')
g = german.groupby('season')

n = dutch.groupby('season')
pol = polish.groupby('season')
por = portuguese.groupby('season')
scot = scots.groupby('season')
switz = swit.groupby('season')
b = belg.groupby('season')

seasons

# 'England','France','Germany','Italy','Spain', 'Netherlands', 'Poland', 'Portugal', 'Scotland', 'Switzerland', 'Belgium'

#New dataframe merging home_team names and matches.
matches_w_teams = pd.merge(new_matches,teams,left_on='home_team_api_id', right_on='team_api_id')
matches_w_teams = matches_w_teams.drop(['id','team_api_id','team_fifa_api_id'],axis = 1)
matches_w_teams = matches_w_teams.rename(columns={'team_long_name':'home_team_long_name','name_y':'league_name','name':'country_name'})
matches_w_teams.head(1)

#New dataframe merging away_team names and matches.
matches_w_teams1 = pd.merge(new_matches,teams,left_on='away_team_api_id', right_on='team_api_id')
matches_w_teams1 = matches_w_teams1.drop(['id','team_api_id','team_fifa_api_id'],axis = 1)
matches_w_teams1 = matches_w_teams1.rename(columns={'team_long_name':'away_team_long_name','name_y':'league_name','name':'country_name'})
matches_w_teams1.head(1)

#Color scheme for each country - from colourbrewer2.org
import matplotlib.patches as mpatches
colors = {'England':'#e41a1c', 'Spain':'#377eb8', 'Italy':'#4daf4a', 'France':'#984ea3', 'Germany':'#ff7f00',
         'Netherlands':'#008000', 'Poland':'#730013', 'Portugal':'#fca400', 'Scotland':'#f7dfb2', 'Switzerland':'#f734ca',
         'Belgium':'#69ffa2'}
color = []

e = mpatches.Patch(color='#e41a1c', label='England')
s = mpatches.Patch(color='#377eb8', label='Spain')
it = mpatches.Patch(color='#4daf4a', label='Italy') #Facepalm note to self : never use i as it'll be used as an iterable for a for loop
f = mpatches.Patch(color='#984ea3', label='France')
g = mpatches.Patch(color='#ff7f00', label='Germany')

n = mpatches.Patch(color='#008000', label='Netherlands')
pol = mpatches.Patch(color='#730013', label='Poland')
por = mpatches.Patch(color='#fca400', label='Portugal')
scot = mpatches.Patch(color='#f7dfb2', label='Scotland')
switz = mpatches.Patch(color='#f734ca', label='Switzerland')
b = mpatches.Patch(color='#69ffa2', label='Belgium')

#Analysing teams in each league
top_goal_scorers = matches_w_teams.groupby('home_team_long_name').total_goals.sum().sort_values(ascending = False)
for i in range(top_n):
    color.append([colors[t] for t in matches_w_teams[matches_w_teams.home_team_long_name == top_goal_scorers.head(top_n).index[i]].country_name.values][0])

top_goal_scorers1 = matches_w_teams1.groupby('away_team_long_name').total_goals.sum().sort_values(ascending = False)
for i in range(top_n):
    color.append([colors[t] for t in matches_w_teams1[matches_w_teams1.away_team_long_name == top_goal_scorers1.head(top_n).index[i]].country_name.values][0])
   

    

##############################################################################################################

###################### Poate sterg

class App(customtkinter.CTk):
    
    WIDTH = 1200
    HEIGHT = 520

    def __init__(self):
        super().__init__()

        self.title("Analiza de Date Sportive")
        self.geometry(f"{App.WIDTH}x{App.HEIGHT}")

        self.protocol("WM_DELETE_WINDOW", self.on_closing)  # call .on_closing() when app gets closed

        # ============ create 4 frames - left, top, bot, right ============

        # configure grid layout (2x1)
        self.grid_columnconfigure(1, weight=1)
        self.grid_rowconfigure(0, weight=1)

        self.frame_left = customtkinter.CTkFrame(master=self, width=180, corner_radius=0)
        self.frame_left.grid(row=0, column=0, rowspan = 2, sticky="nswe")

        self.frame_top = customtkinter.CTkFrame(master=self)
        self.frame_top.grid(row=0, column=1, sticky="nswe", padx=20, pady=20)
        
        self.frame_bot = customtkinter.CTkFrame(master=self)
        self.frame_bot.grid(row=1, column=1, sticky="nswe", padx=20, pady=20)
        
        self.frame_right = customtkinter.CTkFrame(master=self)
        self.frame_right.grid(row=0, column=2, rowspan = 2, sticky="nswe", padx=20, pady=20)

        # ============ frame_left ============

        # configure grid layout (1x11)
        self.frame_left.grid_rowconfigure(0, minsize=10)   # empty row with minsize as spacing
        self.frame_left.grid_rowconfigure(5, weight=1)  # empty row as spacing
        self.frame_left.grid_rowconfigure(8, minsize=20)    # empty row with minsize as spacing
        self.frame_left.grid_rowconfigure(11, minsize=10)  # empty row with minsize as spacing

        self.label_1 = customtkinter.CTkLabel(master=self.frame_left,
                                              text="Tabele Generale")  # font name and size in px
        self.label_1.grid(row=1, column=0, pady=10, padx=10)

        
        self.butonTari = customtkinter.CTkButton(master=self.frame_left, 
                                                 text="Tari", 
                                                 fg_color=("gray75", "gray30"), 
                                                 command = self.Country)
        self.butonTari.grid(row = 2, column = 0, pady=10, padx=20)
        

        self.butonLigi = customtkinter.CTkButton(master=self.frame_left, 
                                         text="Ligi", 
                                         fg_color=("gray75", "gray30"), 
                                         command = self.League)
        self.butonLigi.grid(row = 3, column = 0, pady=10, padx=20)
        
        self.butonMeciuri = customtkinter.CTkButton(master=self.frame_left, 
                                 text="Meciuri", 
                                 fg_color=("gray75", "gray30"), 
                                 command = self.Match)
        self.butonMeciuri.grid(row = 4, column = 0, pady=10, padx=20)
        
        self.butonJucatori = customtkinter.CTkButton(master=self.frame_left, 
                                 text="Jucatori", 
                                 fg_color=("gray75", "gray30"), 
                                 command = self.Player)
        self.butonJucatori.grid(row = 5, column = 0, pady=10, padx=20)
        
        self.butonAtributeJucatori = customtkinter.CTkButton(master=self.frame_left, 
                                 text="Atribute Jucatori", 
                                 fg_color=("gray75", "gray30"), 
                                 command = self.Player_Attributes)
        self.butonAtributeJucatori.grid(row = 6, column = 0, pady=10, padx=20)
        
        self.butonEchipe = customtkinter.CTkButton(master=self.frame_left, 
                                 text="Echipe", 
                                 fg_color=("gray75", "gray30"), 
                                 command = self.Team)
        self.butonEchipe.grid(row = 7, column = 0, pady=10, padx=20)
        
        self.butonAtributeEchipe = customtkinter.CTkButton(master=self.frame_left, 
                                 text="Atribute Echipe", 
                                 fg_color=("gray75", "gray30"), 
                                 command = self.Team_Attributes)
        self.butonAtributeEchipe.grid(row = 8, column = 0, pady=10, padx=20)

        self.switch_dark_mode = customtkinter.CTkSwitch(master=self.frame_left,
                                                text="Dark Mode",
                                                command=self.change_mode)
        self.switch_dark_mode.grid(row=10, column=0, pady=10, padx=20, sticky="w")

        # ============ frame_top ============

        # configure grid layout (3x7)
        self.frame_top.rowconfigure((0, 1, 2, 3), weight=1)
        self.frame_top.rowconfigure(7, weight=10)
        self.frame_top.columnconfigure((0, 1), weight=1)
        self.frame_top.columnconfigure(2, weight=0)
        
        
        self.butonPlayer = customtkinter.CTkButton(master=self.frame_top, 
                                 text="Player Stats", 
                                 fg_color=("gray75", "gray30"), 
                                 command = self.PlayerStats)
        self.butonPlayer.grid(row = 0, column = 0, pady=10, padx=20)
        
        self.entryPlayer = customtkinter.CTkEntry(master=self.frame_top,)
        self.entryPlayer.grid(row = 0, column = 1, pady=10, padx=20)
        
        ###
        self.butonMeciuri = customtkinter.CTkButton(master=self.frame_top, 
                                 text="Matches", 
                                 fg_color=("gray75", "gray30"), 
                                 command = self.Matches)
        self.butonMeciuri.grid(row = 1, column = 0, pady=10, padx=20)
        
        ###
        self.butonMeciuriDupaTara = customtkinter.CTkButton(master=self.frame_top, 
                                 text="Matches by country", 
                                 fg_color=("gray75", "gray30"), 
                                 command = self.MatchesByCountry)
        self.butonMeciuriDupaTara.grid(row = 2, column = 0, pady=10, padx=20)
        
        self.tara = customtkinter.StringVar(value="Choose the League")
        self.taraAleasa = customtkinter.CTkComboBox(master=self.frame_top,
                                     values=["Belgium", "England", "France", "Germany", "Italy", "Netherlands", "Poland", "Portugal", "Scotland", "Spain", "Switzerland"],
                                     variable=self.tara)
        self.taraAleasa.grid(row = 2, column = 1)
        
        ###
        self.butonMeciuriAcasa = customtkinter.CTkButton(master=self.frame_top, 
                                 text="Matches - Home", 
                                 fg_color=("gray75", "gray30"), 
                                 command = self.MatchesHome)
        self.butonMeciuriAcasa.grid(row = 3, column = 0, pady=10, padx=20)
        
        self.entryAcasa = customtkinter.CTkEntry(master=self.frame_top,)
        self.entryAcasa.grid(row = 3, column = 1, pady=10, padx=20)
        
        ###
        self.butonMeciuriDeplasare = customtkinter.CTkButton(master=self.frame_top, 
                                 text="Matches - Away", 
                                 fg_color=("gray75", "gray30"), 
                                 command = self.MatchesAway)
        self.butonMeciuriDeplasare.grid(row = 4, column = 0, pady=10, padx=20)
        
        self.entryDeplasare = customtkinter.CTkEntry(master=self.frame_top,)
        self.entryDeplasare.grid(row = 4, column = 1, pady=10, padx=20)
        
        ###
        self.butonMeciuriAcasaVsDeplasare = customtkinter.CTkButton(master=self.frame_top, 
                                 text="Matches - Home vs Away", 
                                 fg_color=("gray75", "gray30"), 
                                 command = self.MatchesHomeVsAway)
        self.butonMeciuriAcasaVsDeplasare.grid(row = 5, column = 0, pady=10, padx=20)
        
        self.entryAcasaVs = customtkinter.CTkEntry(master=self.frame_top,)
        self.entryAcasaVs.grid(row = 5, column = 1, pady=10, padx=20)
        
        self.entryVsDeplasare = customtkinter.CTkEntry(master=self.frame_top,)
        self.entryVsDeplasare.grid(row = 5, column = 2, pady=10, padx=20)
        

        # ============ frame_bot ============
        
        # ACASA
        
        self.butonGoluriAcasa = customtkinter.CTkButton(master=self.frame_bot, 
                                         text="Top goal scorers - HOME", 
                                         fg_color=("gray75", "gray30"), 
                                         command = self.BestTeamHomeGoals)
        self.butonGoluriAcasa.grid(row = 0, column = 0, pady=10, padx=10)
        
        self.butonGoluriAcasa = customtkinter.CTkButton(master=self.frame_bot, 
                                         text="Team home to loss ratio", 
                                         fg_color=("gray75", "gray30"), 
                                         command = self.Ratio1)
        self.butonGoluriAcasa.grid(row = 0, column = 1, pady=10, padx=10)
        
        self.butonGoluriAcasa = customtkinter.CTkButton(master=self.frame_bot, 
                                         text="Team home to loss ratio", 
                                         fg_color=("gray75", "gray30"), 
                                         command = self.Ratio2)
        self.butonGoluriAcasa.grid(row = 1, column = 0, pady=10, padx=10)
        
        self.butonGoluriAcasa = customtkinter.CTkButton(master=self.frame_bot, 
                                         text="Team home to loss ratio", 
                                         fg_color=("gray75", "gray30"), 
                                         command = self.Ratio3)
        self.butonGoluriAcasa.grid(row = 1, column = 1, pady=10, padx=10)
        
        # DEPLASARE
        
        self.butonGoluriDeplasare = customtkinter.CTkButton(master=self.frame_bot, 
                                         text="Top goal scorers - AWAY", 
                                         fg_color=("gray75", "gray30"), 
                                         command = self.BestTeamAwayGoals)
        self.butonGoluriDeplasare.grid(row = 0, column = 2, pady=10, padx=10)
        
        self.butonGoluriDeplasare = customtkinter.CTkButton(master=self.frame_bot, 
                                         text="Team away to loss ratio", 
                                         fg_color=("gray75", "gray30"), 
                                         command = self.Ratio4)
        self.butonGoluriDeplasare.grid(row = 0, column = 3, pady=10, padx=10)
        
        self.butonGoluriDeplasare = customtkinter.CTkButton(master=self.frame_bot, 
                                         text="Team away to loss ratio", 
                                         fg_color=("gray75", "gray30"), 
                                         command = self.Ratio5)
        self.butonGoluriDeplasare.grid(row = 1, column = 2, pady=10, padx=10)
        
        self.butonGoluriDeplasare = customtkinter.CTkButton(master=self.frame_bot, 
                                         text="Team away to loss ratio", 
                                         fg_color=("gray75", "gray30"), 
                                         command = self.Ratio6)
        self.butonGoluriDeplasare.grid(row = 1, column = 3, pady=10, padx=10)
        

        # ============ frame_right ============
        
        self.butonAverageGoals = customtkinter.CTkButton(master=self.frame_right, 
                                 text="Average Goals", 
                                 fg_color=("gray75", "gray30"), 
                                 command = self.averageGoals)
        self.butonAverageGoals.grid(row = 1, column = 0, pady=10, padx=20)

        self.butonAverageGoalsHome = customtkinter.CTkButton(master=self.frame_right, 
                                 text="Average Goals - Home", 
                                 fg_color=("gray75", "gray30"), 
                                 command = self.averageGoalsHome)
        self.butonAverageGoalsHome.grid(row = 3, column = 0, pady=10, padx=20)
        
        self.butonAverageGoalsAway = customtkinter.CTkButton(master=self.frame_right, 
                                 text="Average Goals - Away", 
                                 fg_color=("gray75", "gray30"), 
                                 command = self.averageGoalsAway)
        self.butonAverageGoalsAway.grid(row = 5, column = 0, pady=10, padx=20)
        
        self.butonAverageGoalsHomeVsAway = customtkinter.CTkButton(master=self.frame_right, 
                                 text="Average Goals - Home vs Away", 
                                 fg_color=("gray75", "gray30"), 
                                 command = self.averageGoalsHomeVsAway)
        self.butonAverageGoalsHomeVsAway.grid(row = 7, column = 0, pady=10, padx=20)
        
        self.butonTotalGoals = customtkinter.CTkButton(master=self.frame_right, 
                                 text="Total Goals", 
                                 fg_color=("gray75", "gray30"), 
                                 command = self.totalGoals)
        self.butonTotalGoals.grid(row = 9, column = 0, pady=10, padx=20)
        
        
        ### CheckBox
        
        self.tari_selectate = []
        self.tari = ('England', 'France', 'Germany', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Scotland', 'Spain', 'Switzerland', "Belgium")
    
        for idx,i in enumerate(self.tari):
            for j in range(3):
                var = StringVar(value=" ")
                customtkinter.CTkCheckBox(master = self.frame_right, text = i, variable = var, command=lambda i = i, var = var: self.process(var, i), onvalue=i).grid(row = idx, column = 1)
            print("\n")

        ### CheckBox
        
        self.butonClasament = customtkinter.CTkButton(master=self.frame_right, 
                                         text="Clasament", 
                                         fg_color=("gray75", "gray30"), 
                                         command = self.Clasament)
        self.butonClasament.grid(row = 11, column = 0, columnspan = 2, pady=10, padx=20)
        
        # Liga
        self.liga = customtkinter.StringVar(value="Choose the League")
        self.ligaAleasa = customtkinter.CTkComboBox(master=self.frame_right,
                                     values=["Belgium", "England", "France", "Germany", "Italy", "Netherlands", "Poland", "Portugal", "Scotland", "Spain", "Switzerland"],
                                     variable=self.liga)
        self.ligaAleasa.grid(row = 12, column = 0)


        # Anul
        self.an = customtkinter.StringVar(value="Choose the Year")
        self.anAles = customtkinter.CTkComboBox(master=self.frame_right,
                                     values=['2008/2009', '2009/2010','2010/2011','2011/2012', '2012/2013','2013/2014','2014/2015','2015/2016'],
                                     variable=self.an)
        self.anAles.grid(row = 12, column = 1)
        
        
        
        # set default values
        self.switch_dark_mode.select()

    def process(self, var, text):
        try:
            val = int(var.get()) # If not selected it will give 0 as int, which will trigger `else` block
        except ValueError:
            val = var.get()

        if val: # if val is not empty, ie, if val is any selected value
            self.tari_selectate.append(val)
        else: # if val is 0 
            self.tari_selectate.remove(text) # Remove the corresponding text from the list
        # print(slct_ctry_lst)
        
    def res(self):
        if row['home_team_goal'] == row['away_team_goal']:
            val = 0
        elif row['home_team_goal'] > row['away_team_goal']:
            val = 1
        else:
            val = -1
        return val

    def autolabel(self, rects):
        # attach some text labels
        for rect in rects:
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width()/2., 1*height, '%d' % int(height), ha='center', va='bottom')
        
    def Country(self):
        conn = sqlite3.connect('database.sqlite')
        countries = pd.read_sql("""SELECT * FROM Country;""", conn)

        toplevel = customtkinter.CTkToplevel(app)
        toplevel.geometry("350x240+800+300")
        displayData = pt.Table(toplevel, dataframe=countries, showtoolbar=False, showstatusbar=True)
        displayData.show()
        
    def League(self):
        conn = sqlite3.connect('database.sqlite')
        leagues = pd.read_sql("""SELECT League.name, Country.name
                                FROM League
                                JOIN Country ON Country.id = League.country_id;""", conn)

        toplevel = customtkinter.CTkToplevel(app)
        toplevel.geometry("350x240+800+300")
        displayData = pt.Table(toplevel, dataframe=leagues, showtoolbar=False, showstatusbar=True)
        displayData.show()

    def Match(self):
        conn = sqlite3.connect('database.sqlite')
        matches = pd.read_sql("""SELECT * FROM Match""", conn)

        toplevel = customtkinter.CTkToplevel(app)
        toplevel.geometry("350x240+800+300")
        displayData = pt.Table(toplevel, dataframe=matches, showtoolbar=False, showstatusbar=True)
        displayData.show()

    def Player(self):
        conn = sqlite3.connect('database.sqlite')
        players = pd.read_sql("""SELECT * FROM Player;""", conn)

        toplevel = customtkinter.CTkToplevel(app)
        toplevel.geometry("350x240+800+300")
        displayData = pt.Table(toplevel, dataframe=players, showtoolbar=False, showstatusbar=True)
        displayData.show()

    def Player_Attributes(self):
        conn = sqlite3.connect('database.sqlite')
        players_atr = pd.read_sql("""SELECT * FROM Player_Attributes;""", conn)

        toplevel = customtkinter.CTkToplevel(app)
        toplevel.geometry("350x240+800+300")
        displayData = pt.Table(toplevel, dataframe=players_atr, showtoolbar=False, showstatusbar=True)
        displayData.show()

    def Team(self):
        conn = sqlite3.connect('database.sqlite')
        teams = pd.read_sql("""SELECT * FROM Team;""", conn)

        toplevel = customtkinter.CTkToplevel(app)
        toplevel.geometry("350x240+800+300")
        displayData = pt.Table(toplevel, dataframe=teams, showtoolbar=False, showstatusbar=True)
        displayData.show()

    def Team_Attributes(self):
        conn = sqlite3.connect('database.sqlite')
        team_atr = pd.read_sql("""SELECT * FROM Team_Attributes;""", conn)

        toplevel = customtkinter.CTkToplevel(app)
        toplevel.geometry("350x240+800+300")
        displayData = pt.Table(toplevel, dataframe=team_atr, showtoolbar=False, showstatusbar=True)
        displayData.show()
        
    def Clasament(self):
        r1 = self.liga.get()
        r2 = self.an.get()

        conn = sqlite3.connect('database.sqlite')
        league = pd.read_sql('''SELECT home.team_long_name AS homeTeam,
           match.home_team_goal,
           (CASE
                WHEN home_team_goal = away_team_goal THEN 1
                WHEN home_team_goal > away_team_goal THEN 3
                WHEN home_team_goal < away_team_goal THEN 0
            END)pointHome,
           away.team_long_name AS awayTeam,
           Match.away_team_goal,
           (CASE
                WHEN home_team_goal = away_team_goal THEN 1
                WHEN home_team_goal > away_team_goal THEN 0
                WHEN home_team_goal < away_team_goal THEN 3
            END)pointAway
        FROM Country
        JOIN MATCH ON match.country_id = country.id
        AND name = :r1
        AND season = :r2
        LEFT JOIN Team AS home ON home.team_api_id = Match.home_team_api_id
        LEFT JOIN Team AS away ON away.team_api_id = Match.away_team_api_id
        ;''', conn, params = {'r1':r1, 'r2':r2})

        league1 = league[['homeTeam', 'home_team_goal', 'pointHome']]
        league2 = league[['awayTeam', 'away_team_goal', 'pointAway']]

        totalleague = pd.DataFrame(np.concatenate((
            league1.values, league2.values), axis=0))
        totalleague.columns = ['team', 'goals', 'points']

        leagueSummary = totalleague.groupby('team', as_index=False)[['goals', 'points']].sum()
        standing = leagueSummary.sort_values(by='points', ascending=False)

        toplevel = customtkinter.CTkToplevel(app)
        toplevel.geometry("350x240+800+300")
        toplevel.title("Clasament")
        displayData = pt.Table(toplevel, dataframe=standing, showtoolbar=False, showstatusbar=True)
        displayData.show()

    def averageGoalsHome(self):
        %matplotlib qt

        conn = sqlite3.connect('database.sqlite')
        leages_by_season = pd.read_sql("""SELECT Country.name AS country_name, 
                                                League.name AS league_name, 
                                                season,
                                                count(distinct stage) AS number_of_stages,
                                                count(distinct HT.team_long_name) AS number_of_teams,
                                                avg(home_team_goal) AS avg_home_team_scors, 
                                                avg(away_team_goal) AS avg_away_team_goals, 
                                                avg(home_team_goal-away_team_goal) AS avg_goal_dif, 
                                                avg(home_team_goal+away_team_goal) AS avg_goals, 
                                                sum(home_team_goal+away_team_goal) AS total_goals                                       
                                        FROM Match
                                        JOIN Country on Country.id = Match.country_id
                                        JOIN League on League.id = Match.league_id
                                        LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                        LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                        WHERE country_name in ('Belgium', 'England', 'France', 'Germany', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Scotland', 'Spain', 'Switzerland')
                                        GROUP BY Country.name, League.name, season
                                        HAVING count(distinct stage) > 10
                                        ORDER BY Country.name, League.name, season DESC
                                        ;""", conn)


        df = pd.DataFrame(index=np.sort(leages_by_season['season'].unique()), columns=leages_by_season['country_name'].unique())

        for i in self.tari_selectate:
            if i in self.tari_selectate:
                df.loc[:,i] = list(leages_by_season.loc[leages_by_season['country_name']==i,'avg_home_team_scors'])

        df.plot(figsize=(14,9),title='Average Goals - Home Team')

        widgetTopLevel = tk.Toplevel()
        widgetTopLevel.title('Average Goals - Home Team')
        displayData = pt.Table(widgetTopLevel, dataframe=df, showtoolbar=False, showstatusbar=True)
        displayData.show()

    def averageGoalsAway(self):
        %matplotlib qt

        conn = sqlite3.connect('database.sqlite')
        leages_by_season = pd.read_sql("""SELECT Country.name AS country_name, 
                                                League.name AS league_name, 
                                                season,
                                                count(distinct stage) AS number_of_stages,
                                                count(distinct HT.team_long_name) AS number_of_teams,
                                                avg(home_team_goal) AS avg_home_team_scors, 
                                                avg(away_team_goal) AS avg_away_team_goals, 
                                                avg(home_team_goal-away_team_goal) AS avg_goal_dif, 
                                                avg(home_team_goal+away_team_goal) AS avg_goals, 
                                                sum(home_team_goal+away_team_goal) AS total_goals                                       
                                        FROM Match
                                        JOIN Country on Country.id = Match.country_id
                                        JOIN League on League.id = Match.league_id
                                        LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                        LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                        WHERE country_name in ('Belgium', 'England', 'France', 'Germany', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Scotland', 'Spain', 'Switzerland')
                                        GROUP BY Country.name, League.name, season
                                        HAVING count(distinct stage) > 10
                                        ORDER BY Country.name, League.name, season DESC
                                        ;""", conn)


        df = pd.DataFrame(index=np.sort(leages_by_season['season'].unique()), columns=leages_by_season['country_name'].unique())

        for i in self.tari_selectate:
            if i in self.tari_selectate:
                df.loc[:,i] = list(leages_by_season.loc[leages_by_season['country_name']==i,'avg_away_team_goals'])

        df.plot(figsize=(14,9),title='Average Goals - Away Team')

        widgetTopLevel = tk.Toplevel()
        widgetTopLevel.title('Average Goals - Away Team')
        displayData = pt.Table(widgetTopLevel, dataframe=df, showtoolbar=False, showstatusbar=True)
        displayData.show()

    def averageGoals(self):
        %matplotlib qt

        conn = sqlite3.connect('database.sqlite')
        leages_by_season = pd.read_sql("""SELECT Country.name AS country_name, 
                                                League.name AS league_name, 
                                                season,
                                                count(distinct stage) AS number_of_stages,
                                                count(distinct HT.team_long_name) AS number_of_teams,
                                                avg(home_team_goal) AS avg_home_team_scors, 
                                                avg(away_team_goal) AS avg_away_team_goals, 
                                                avg(home_team_goal-away_team_goal) AS avg_goal_dif, 
                                                avg(home_team_goal+away_team_goal) AS avg_goals, 
                                                sum(home_team_goal+away_team_goal) AS total_goals                                       
                                        FROM Match
                                        JOIN Country on Country.id = Match.country_id
                                        JOIN League on League.id = Match.league_id
                                        LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                        LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                        WHERE country_name in ('Belgium', 'England', 'France', 'Germany', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Scotland', 'Spain', 'Switzerland')
                                        GROUP BY Country.name, League.name, season
                                        HAVING count(distinct stage) > 10
                                        ORDER BY Country.name, League.name, season DESC
                                        ;""", conn)


        df = pd.DataFrame(index=np.sort(leages_by_season['season'].unique()), columns=leages_by_season['country_name'].unique())

        for i in self.tari_selectate:
            if i in self.tari_selectate:
                df.loc[:,i] = list(leages_by_season.loc[leages_by_season['country_name']==i,'avg_goals'])

        df.plot(figsize=(14,9),title='Average Goals per Game Over Time')

        widgetTopLevel = tk.Toplevel()
        widgetTopLevel.title('Goals Over Time')
        displayData = pt.Table(widgetTopLevel, dataframe=df, showtoolbar=False, showstatusbar=True)
        displayData.show()

    def averageGoalsHomeVsAway(self):
        %matplotlib qt

        conn = sqlite3.connect('database.sqlite')
        leages_by_season = pd.read_sql("""SELECT Country.name AS country_name, 
                                                League.name AS league_name, 
                                                season,
                                                count(distinct stage) AS number_of_stages,
                                                count(distinct HT.team_long_name) AS number_of_teams,
                                                avg(home_team_goal) AS avg_home_team_scors, 
                                                avg(away_team_goal) AS avg_away_team_goals, 
                                                avg(home_team_goal-away_team_goal) AS avg_goal_dif, 
                                                avg(home_team_goal+away_team_goal) AS avg_goals, 
                                                sum(home_team_goal+away_team_goal) AS total_goals                                       
                                        FROM Match
                                        JOIN Country on Country.id = Match.country_id
                                        JOIN League on League.id = Match.league_id
                                        LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                        LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                        WHERE country_name in ('Belgium', 'England', 'France', 'Germany', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Scotland', 'Spain', 'Switzerland')
                                        GROUP BY Country.name, League.name, season
                                        HAVING count(distinct stage) > 10
                                        ORDER BY Country.name, League.name, season DESC
                                        ;""", conn)

        df = pd.DataFrame(index=np.sort(leages_by_season['season'].unique()), columns=leages_by_season['country_name'].unique())

        for i in self.tari_selectate:
            if i in self.tari_selectate:
                df.loc[:,i] = list(leages_by_season.loc[leages_by_season['country_name']==i,'avg_goal_dif'])

        df.plot(figsize=(14,9),title='Average Goals Difference Home vs Out')

        widgetTopLevel = tk.Toplevel()
        widgetTopLevel.title('Goals Over Time')
        displayData = pt.Table(widgetTopLevel, dataframe=df, showtoolbar=False, showstatusbar=True)
        displayData.show()

    def totalGoals(self):

        %matplotlib qt

        conn = sqlite3.connect('database.sqlite')
        leages_by_season = pd.read_sql("""SELECT Country.name AS country_name, 
                                                League.name AS league_name, 
                                                season,
                                                count(distinct stage) AS number_of_stages,
                                                count(distinct HT.team_long_name) AS number_of_teams,
                                                avg(home_team_goal) AS avg_home_team_scors, 
                                                avg(away_team_goal) AS avg_away_team_goals, 
                                                avg(home_team_goal-away_team_goal) AS avg_goal_dif, 
                                                avg(home_team_goal+away_team_goal) AS avg_goals, 
                                                sum(home_team_goal+away_team_goal) AS total_goals                                       
                                        FROM Match
                                        JOIN Country on Country.id = Match.country_id
                                        JOIN League on League.id = Match.league_id
                                        LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                        LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                        WHERE country_name in ('Belgium', 'England', 'France', 'Germany', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Scotland', 'Spain', 'Switzerland')
                                        GROUP BY Country.name, League.name, season
                                        HAVING count(distinct stage) > 10
                                        ORDER BY Country.name, League.name, season DESC
                                        ;""", conn)

        df = pd.DataFrame(index=np.sort(leages_by_season['season'].unique()), columns=leages_by_season['country_name'].unique())

        for i in self.tari_selectate:
            if i in self.tari_selectate:
                df.loc[:,i] = list(leages_by_season.loc[leages_by_season['country_name']==i,'total_goals'])

        df.plot(figsize=(14,9),title='Average Goals Difference Home vs Out')

        widgetTopLevel = tk.Toplevel()
        widgetTopLevel.title('Goals Over Time')
        displayData = pt.Table(widgetTopLevel, dataframe=df, showtoolbar=False, showstatusbar=True)
        displayData.show()
        
    def PlayerStats(self):
        conn = sqlite3.connect('database.sqlite')
        
        player = self.entryPlayer.get()
        playerskills = pd.read_sql("""SELECT a.player_api_id,player_name ,date,birthday,overall_rating,potential,crossing,
                                            finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,
                                            long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,
                                            shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,
                                            vision,penalties,marking,standing_tackle,sliding_tackle  
                                      FROM Player_Attributes a join player b on a.player_api_id = b.player_api_id
                                      WHERE player_name = :player;""", conn , params = {'player':player})
        
        toplevel = customtkinter.CTkToplevel(app)
        toplevel.geometry("350x240+800+300")
        displayData = pt.Table(toplevel, dataframe = playerskills, showtoolbar=False, showstatusbar=True)
        displayData.show()
        
    def Matches(self):
        conn = sqlite3.connect('database.sqlite')
        meciuri = pd.read_sql("""SELECT Match.id, 
                                    Country.name AS country_name, 
                                    League.name AS league_name, 
                                    season, 
                                    stage, 
                                    date,
                                    HT.team_long_name AS  home_team,
                                    AT.team_long_name AS away_team,
                                    home_team_goal, 
                                    away_team_goal                                        
                                    FROM Match
                                    JOIN Country on Country.id = Match.country_id
                                    JOIN League on League.id = Match.league_id
                                    LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                    LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                    ORDER by country_name;""", conn)

        toplevel = customtkinter.CTkToplevel(app)
        toplevel.geometry("350x240+800+300")
        displayData = pt.Table(toplevel, dataframe = meciuri, showtoolbar=False, showstatusbar=True)
        displayData.show()
        
    def MatchesByCountry(self):
        conn = sqlite3.connect('database.sqlite')
        t1 = self.tara.get()
        meciuriDupaTara = pd.read_sql("""SELECT Match.id, 
                            Country.name AS country_name, 
                            League.name AS league_name, 
                            season, 
                            stage, 
                            date,
                            HT.team_long_name AS  home_team,
                            AT.team_long_name AS away_team,
                            home_team_goal, 
                            away_team_goal                                        
                            FROM Match
                            JOIN Country on Country.id = Match.country_id
                            JOIN League on League.id = Match.league_id
                            LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                            LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                            WHERE country_name = :t1
                            ORDER by date;""", conn, params = {'t1':t1})

        toplevel = customtkinter.CTkToplevel(app)
        toplevel.geometry("350x240+800+300")
        displayData = pt.Table(toplevel, dataframe = meciuriDupaTara, showtoolbar=False, showstatusbar=True)
        displayData.show()
        
    def MatchesHome(self):
        conn = sqlite3.connect('database.sqlite')
        
        echipa = self.entryAcasa.get()
        meciAcasa = pd.read_sql("""SELECT Match.id, Match.match_api_id, 
                            Country.name AS country_name, 
                            League.name AS league_name, 
                            season, 
                            stage, 
                            date,
                            HT.team_long_name AS  home_team,
                            AT.team_long_name AS away_team,
                            home_team_goal, 
                            away_team_goal                                        
                            FROM Match
                            JOIN Country on Country.id = Match.country_id
                            JOIN League on League.id = Match.league_id
                            LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                            LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                            WHERE home_team = :echipa
                            ORDER by date;""", conn, params = {'echipa':echipa})
        
        toplevel = customtkinter.CTkToplevel(app)
        toplevel.geometry("350x240+800+300")
        displayData = pt.Table(toplevel, dataframe = meciAcasa, showtoolbar=False, showstatusbar=True)
        displayData.show()
        
    def MatchesAway(self):
        conn = sqlite3.connect('database.sqlite')
        
        echipa = self.entryDeplasare.get()
        meciDeplasare = pd.read_sql("""SELECT Match.id, 
                            Country.name AS country_name, 
                            League.name AS league_name, 
                            season, 
                            stage, 
                            date,
                            HT.team_long_name AS  home_team,
                            AT.team_long_name AS away_team,
                            home_team_goal, 
                            away_team_goal                                        
                            FROM Match
                            JOIN Country on Country.id = Match.country_id
                            JOIN League on League.id = Match.league_id
                            LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                            LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                            WHERE home_team = :echipa
                            ORDER by date;""", conn, params = {'echipa':echipa})
        
        toplevel = customtkinter.CTkToplevel(app)
        toplevel.geometry("350x240+800+300")
        displayData = pt.Table(toplevel, dataframe = meciDeplasare, showtoolbar=False, showstatusbar=True)
        displayData.show()
        
    def MatchesHomeVsAway(self):
        conn = sqlite3.connect('database.sqlite')
        
        echipa1 = self.entryAcasaVs.get()
        echipa2 = self.entryVsDeplasare.get()
        meciAcasaVsDeplasare = pd.read_sql("""SELECT Match.id, 
                            Country.name AS country_name, 
                            League.name AS league_name, 
                            season, 
                            stage, 
                            date,
                            HT.team_long_name AS  home_team,
                            AT.team_long_name AS away_team,
                            home_team_goal, 
                            away_team_goal                                        
                            FROM Match
                            JOIN Country on Country.id = Match.country_id
                            JOIN League on League.id = Match.league_id
                            LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                            LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                            WHERE home_team = :echipa1 AND away_team = :echipa2
                            ORDER by date;""", conn, params = {'echipa1':echipa1, 'echipa2':echipa2})
        
        toplevel = customtkinter.CTkToplevel(app)
        toplevel.geometry("350x240+800+300")
        displayData = pt.Table(toplevel, dataframe = meciAcasaVsDeplasare, showtoolbar=False, showstatusbar=True)
        displayData.show()

    def BestTeamHomeGoals(self):
        %matplotlib qt

        fig = plt.figure(figsize = (10,8))
        ax = fig.add_subplot(1,1,1)
        rects = ax.bar(range(top_n), top_goal_scorers.head(top_n).values,align = "center",color = color)

        ax.set_xticks(range(top_n))
        ax.set_xticklabels(top_goal_scorers.head(top_n).index,rotation = "vertical")
        ax.set_title("Top goal scorers at home")
        ax.set_ylabel("Number of goals")
        ax.set_xlabel("Team name")

        ax.legend([e, s, it, f, g, n, pol, por, scot, switz, b], colors.keys())
            # attach some text labels
        for rect in rects:
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width()/2., 1*height, '%d' % int(height), ha='center', va='bottom')
        
    def Ratio1(self):
        %matplotlib qt

        fig = plt.figure(figsize = (10,8))
        plt.style.use('ggplot')
        colorwl = []
        colorwld = []
        colorwdl = []
        for i in range(top_n):
            colorwl.append([colors[t] for t in matches_w_teams[matches_w_teams.home_team_long_name == team_home_wl_ratio.head(top_n).index[i]].country_name.values][0])
            colorwld.append([colors[t] for t in matches_w_teams[matches_w_teams.home_team_long_name == team_home_wld_ratio.head(top_n).index[i]].country_name.values][0])
            colorwdl.append([colors[t] for t in matches_w_teams[matches_w_teams.home_team_long_name == team_home_wdl_ratio.head(top_n).index[i]].country_name.values][0])

        ax1 = fig.add_subplot(1,1,1)
        rects1 = ax1.bar(range(top_n), team_home_wl_ratio.head(top_n).values,align = "center", color = colorwl)
        ax1.set_xticks(range(top_n))
        ax1.set_xticklabels(team_home_wl_ratio.head(top_n).index,rotation = "vertical")
        ax1.set_title("Team home to loss ratio (without draws)")
        ax1.set_ylabel("Win to Loss ratio")
        ax1.set_xlabel("Team names")
        ax1.legend([e, s, it, f, g, n, pol, por, scot, switz, b], ['England', 'Spain','Italy','France','Germany', 'Netherlands', 'Poland', 'Portugal', 'Scotland', 'Switzerland', 'Belgium'])
        plt.tight_layout()

    def Ratio2(self):
        %matplotlib qt

        fig = plt.figure(figsize = (10,8))
        plt.style.use('ggplot')
        colorwl = []
        colorwld = []
        colorwdl = []
        for i in range(top_n):
            colorwl.append([colors[t] for t in matches_w_teams[matches_w_teams.home_team_long_name == team_home_wl_ratio.head(top_n).index[i]].country_name.values][0])
            colorwld.append([colors[t] for t in matches_w_teams[matches_w_teams.home_team_long_name == team_home_wld_ratio.head(top_n).index[i]].country_name.values][0])
            colorwdl.append([colors[t] for t in matches_w_teams[matches_w_teams.home_team_long_name == team_home_wdl_ratio.head(top_n).index[i]].country_name.values][0])

        ax2 = fig.add_subplot(1,1,1)
        rects2 = ax2.bar(range(top_n), team_home_wld_ratio.head(top_n).values,align = "center", color = colorwld)
        ax2.set_xticks(range(top_n))
        ax2.set_xticklabels(team_home_wld_ratio.head(top_n).index,rotation = "vertical")
        ax2.set_title("Team home to loss ratio (with draws considered as a loss)")
        ax2.set_ylabel("Win to (Loss or Draw) ratio")
        ax2.set_xlabel("Team names")
        ax2.legend([e, s, it, f, g, n, pol, por, scot, switz, b], ['England', 'Spain','Italy','France','Germany', 'Netherlands', 'Poland', 'Portugal', 'Scotland', 'Switzerland', 'Belgium'])

    def Ratio3(self):
        %matplotlib qt

        fig = plt.figure(figsize = (10,8))
        plt.style.use('ggplot')
        colorwl = []
        colorwld = []
        colorwdl = []
        for i in range(top_n):
            colorwl.append([colors[t] for t in matches_w_teams[matches_w_teams.home_team_long_name == team_home_wl_ratio.head(top_n).index[i]].country_name.values][0])
            colorwld.append([colors[t] for t in matches_w_teams[matches_w_teams.home_team_long_name == team_home_wld_ratio.head(top_n).index[i]].country_name.values][0])
            colorwdl.append([colors[t] for t in matches_w_teams[matches_w_teams.home_team_long_name == team_home_wdl_ratio.head(top_n).index[i]].country_name.values][0])

        ax3 = fig.add_subplot(1,1,1)
        rects3 = ax3.bar(range(top_n), team_home_wdl_ratio.head(top_n).values,align = "center", color = colorwdl)
        ax3.set_xticks(range(top_n))
        ax3.set_xticklabels(team_home_wdl_ratio.head(top_n).index,rotation = "vertical")
        ax3.set_title("Team home to loss ratio (with draws considered as a loss)")
        ax3.set_ylabel("(Win or draw) to loss ratio")
        ax3.set_xlabel("Team names")
        ax3.legend([e, s, it, f, g, n, pol, por, scot, switz, b], ['England', 'Spain','Italy','France','Germany', 'Netherlands', 'Poland', 'Portugal', 'Scotland', 'Switzerland', 'Belgium'])
        plt.tight_layout()
        
    def BestTeamAwayGoals(self):
        %matplotlib qt

        fig = plt.figure(figsize = (10,8))
        ax = fig.add_subplot(1,1,1)
        rects = ax.bar(range(top_n), top_goal_scorers1.head(top_n).values,align = "center",color = color)

        ax.set_xticks(range(top_n))
        ax.set_xticklabels(top_goal_scorers1.head(top_n).index,rotation = "vertical")
        ax.set_title("Top goal scorers at away")
        ax.set_ylabel("Number of goals")
        ax.set_xlabel("Team name")

        ax.legend([e, s, it, f, g, n, pol, por, scot, switz, b], colors.keys())
            # attach some text labels
        for rect in rects:
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width()/2., 1*height, '%d' % int(height), ha='center', va='bottom')

    def Ratio4(self):
        %matplotlib qt

        fig = plt.figure(figsize = (10,8))
        plt.style.use('ggplot')
        colorwl = []
        colorwld = []
        colorwdl = []
        for i in range(top_n):
            colorwl.append([colors[t] for t in matches_w_teams1[matches_w_teams1.away_team_long_name == team_away_wl_ratio.head(top_n).index[i]].country_name.values][0])
            colorwld.append([colors[t] for t in matches_w_teams1[matches_w_teams1.away_team_long_name == team_away_wld_ratio.head(top_n).index[i]].country_name.values][0])
            colorwdl.append([colors[t] for t in matches_w_teams1[matches_w_teams1.away_team_long_name == team_away_wdl_ratio.head(top_n).index[i]].country_name.values][0])


        ax1 = fig.add_subplot(1,1,1)
        rects1 = ax1.bar(range(top_n), team_away_wl_ratio.head(top_n).values,align = "center", color = colorwl)
        ax1.set_xticks(range(top_n))
        ax1.set_xticklabels(team_away_wl_ratio.head(top_n).index,rotation = "vertical")
        ax1.set_title("Team away to loss ratio (without draws)")
        ax1.set_ylabel("Win to Loss ratio")
        ax1.set_xlabel("Team names")
        ax1.legend([e, s, it, f, g, n, pol, por, scot, switz, b], ['England', 'Spain','Italy','France','Germany', 'Netherlands', 'Poland', 'Portugal', 'Scotland', 'Switzerland', 'Belgium'])
        plt.tight_layout()

    def Ratio5(self):
        %matplotlib qt

        fig = plt.figure(figsize = (10,8))
        plt.style.use('ggplot')
        colorwl = []
        colorwld = []
        colorwdl = []
        for i in range(top_n):
            colorwl.append([colors[t] for t in matches_w_teams1[matches_w_teams1.away_team_long_name == team_away_wl_ratio.head(top_n).index[i]].country_name.values][0])
            colorwld.append([colors[t] for t in matches_w_teams1[matches_w_teams1.away_team_long_name == team_away_wld_ratio.head(top_n).index[i]].country_name.values][0])
            colorwdl.append([colors[t] for t in matches_w_teams1[matches_w_teams1.away_team_long_name == team_away_wdl_ratio.head(top_n).index[i]].country_name.values][0])

        ax2 = fig.add_subplot(1,1,1)
        rects2 = ax2.bar(range(top_n), team_away_wld_ratio.head(top_n).values,align = "center", color = colorwld)
        ax2.set_xticks(range(top_n))
        ax2.set_xticklabels(team_away_wld_ratio.head(top_n).index,rotation = "vertical")
        ax2.set_title("Team away to loss ratio (with draws considered as a loss)")
        ax2.set_ylabel("Win to (Loss or Draw) ratio")
        ax2.set_xlabel("Team names")
        ax2.legend([e, s, it, f, g, n, pol, por, scot, switz, b], ['England', 'Spain','Italy','France','Germany', 'Netherlands', 'Poland', 'Portugal', 'Scotland', 'Switzerland', 'Belgium'])

    def Ratio6(self):
        %matplotlib qt

        fig = plt.figure(figsize = (10,8))
        plt.style.use('ggplot')
        colorwl = []
        colorwld = []
        colorwdl = []
        for i in range(top_n):
            colorwl.append([colors[t] for t in matches_w_teams1[matches_w_teams1.away_team_long_name == team_away_wl_ratio.head(top_n).index[i]].country_name.values][0])
            colorwld.append([colors[t] for t in matches_w_teams1[matches_w_teams1.away_team_long_name == team_away_wld_ratio.head(top_n).index[i]].country_name.values][0])
            colorwdl.append([colors[t] for t in matches_w_teams1[matches_w_teams1.away_team_long_name == team_away_wdl_ratio.head(top_n).index[i]].country_name.values][0])

        ax3 = fig.add_subplot(1,1,1)
        rects3 = ax3.bar(range(top_n), team_away_wdl_ratio.head(top_n).values,align = "center", color = colorwdl)
        ax3.set_xticks(range(top_n))
        ax3.set_xticklabels(team_away_wdl_ratio.head(top_n).index,rotation = "vertical")
        ax3.set_title("Team away to loss ratio (with draws considered as a loss)")
        ax3.set_ylabel("(Win or draw) to loss ratio")
        ax3.set_xlabel("Team names")
        ax3.legend([e, s, it, f, g, n, pol, por, scot, switz, b], ['England', 'Spain','Italy','France','Germany', 'Netherlands', 'Poland', 'Portugal', 'Scotland', 'Switzerland', 'Belgium'])
        plt.tight_layout()
        
    def change_mode(self):
        if self.switch_dark_mode.get() == 1:
            customtkinter.set_appearance_mode("dark")
        else:
            customtkinter.set_appearance_mode("light")

    def on_closing(self, event=0):
        self.destroy()

    def start(self):
        self.mainloop()

if __name__ == "__main__":
    app = App()
    app.start()